In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic'
df = pd.read_csv('결측치제거_l (1).csv', encoding='cp949')

In [6]:
df

,기준년월,가명 식별자,업종(중분류),외감구분,설립일자,종업원수,주소지시군구,유동자산,비유동자산,당좌자산,...,기준년월 시점 신용도판단정보 및 공공정보 (한국신용정보원) 최근 해제일자로 부터 경과일수,10구간으로 구분(1~10. 1등급일수록 우량),"기준일로부터 향후 1년내 부도, 기업회생, 90일이상 금융연체, 대지급 등 발생여부(신용정보원 기준). 1: 향후1년내 부도발생, 0: 향후1년내 부도미발생",업종(대분류),대분류_코드,중분류_코드,시도코드,시도명,시군구코드,시군구명
0,20210801,125539,"전기, 가스, 증기 및 공기 조절 공급업",1,19610624,23534,11680,18824052701,219366858158,18411820858,...,999999999,4,0,"전기, 가스, 증기 및 공기 조절 공급업",D,35,11,서울,680,강남구
1,20210801,13475,"전기, 가스, 증기 및 공기 조절 공급업",1,19610816,23904,11680,18567715519,202170306831,18171434000,...,999999999,4,0,"전기, 가스, 증기 및 공기 조절 공급업",D,35,11,서울,680,강남구
2,20210801,149275,자동차 및 트레일러 제조업,1,19660923,72689,11650,47263779547,109085622838,41300322000,...,999999999,2,0,제조업,C,30,11,서울,650,서초구
3,20210801,21529,"전기, 가스, 증기 및 공기 조절 공급업",1,20010730,12716,47130,14976938599,112647498908,4259050008,...,999999999,4,0,"전기, 가스, 증기 및 공기 조절 공급업",D,35,47,경북,130,경주시
4,20210801,7713,"전기, 가스, 증기 및 공기 조절 공급업",1,19610406,11531,11680,37708470980,70966188895,29690186759,...,999999999,2,0,"전기, 가스, 증기 및 공기 조절 공급업",D,35,11,서울,680,강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124984,20210801,138647,기타 개인 서비스업,2,20180401,3,11290,257,0,257,...,173,6,0,"협회 및 단체, 수리 및 기타 개인 서비스업",S,96,11,서울,290,성북구
124985,20210801,140704,기타 운송장비 제조업,2,20180803,13,48310,0,0,0,...,999999999,4,0,제조업,C,31,48,경남,310,거제시
124986,20210801,114892,사업 지원 서비스업,2,20200219,9,46130,2681,-9435,2681,...,999999999,4,0,"사업시설 관리, 사업 지원 및 임대 서비스업",N,75,46,전남,130,여수시
124987,20210801,27640,부동산업,1,20170206,2,11215,36067348,-101019284,21505869,...,999999999,7,0,부동산업,L,68,11,서울,215,광진구


In [7]:
df_건설 = df[df['업종(대분류)'].str.contains('건설', na=False)]

In [8]:
df_건설.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15653 entries, 13 to 124943
Columns: 165 entries, 기준년월 to 시군구명
dtypes: float64(38), int64(122), object(5)
memory usage: 19.8+ MB


In [27]:
# object 타입 컬럼 목록
object_cols = df_건설.select_dtypes(include='object').columns.tolist()

# 고유값 수 확인
for col in object_cols:
    print(f"{col}: {df_건설[col].nunique()}개 → 예시: {df_건설[col].unique()[:5]}")


업종(중분류): 2개 → 예시: ['종합 건설업' '전문직별 공사업']
업종(대분류): 1개 → 예시: ['건설업']
대분류_코드: 1개 → 예시: ['F']
시도명: 16개 → 예시: ['서울' '경기' '광주' '부산' '전남']
시군구명: 188개 → 예시: ['종로구' '용산구' '중구' '강남구' '고양시']


In [ ]:
encoding_targets = ['업종(중분류)', '시도명']

In [29]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df_건설.copy()
encoders = {}

for col in ['업종(중분류)', '시도명']:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    encoders[col] = le  # 나중에 해석할 수 있게 저장


In [30]:
# 타깃 분리
y = df_encoded['기준일로부터 향후 1년내 부도, 기업회생, 90일이상 금융연체, 대지급 등 발생여부(신용정보원 기준). 1: 향후1년내 부도발생, 0: 향후1년내 부도미발생']

# 수치형 + 인코딩된 컬럼 전체 목록
feature_cols = df_encoded.select_dtypes(include=['int64', 'float64']).drop(columns=[y.name]).columns

# 상관관계 계산
correlations = df_encoded[feature_cols].corrwith(y).sort_values(key=abs, ascending=False)

# 상위 30개 출력
print("📌 타깃(y)과 상관관계 높은 피처 TOP 30:\n", correlations.head(30))


📌 타깃(y)과 상관관계 높은 피처 TOP 30:
 10구간으로 구분(1~10. 1등급일수록 우량)                                  0.136452
기준년월 시점 3개월내 연체일수가 30일 이상 된 대출연체 유지 과목수(연체대출금 상환 포함)        0.079205
기준년월 시점 6개월내 대출연체 발생분 최장 보유일수(연체대출금 상환 포함)                  0.076730
기준년월 시점 신용도판단정보 및 공공정보 (한국신용정보원) 최근 해제일자로 부터 경과일수          -0.071204
기준년월 시점 신용도판단정보 및 공공정보 (한국신용정보원) 최근 발생일자로 부터 경과일수(해제 포함)   -0.071204
기준년월 시점 1년내 대출연체 발생분 최장 보유일수(연체대출금 상환 포함)                   0.070084
기준년월 시점 6개월내 연체일수가 30일 이상 된 대출연체 유지 기관수(연체대출금 상환 포함)        0.069672
기준년월 시점 공공정보 (한국신용정보원) 유지건수(해제 제외)                          0.068821
기준년월 시점 3개월내 연체일수가 30일 이상 된 대출연체 유지 기관수(연체대출금 상환 포함)        0.067734
기준년월 시점 1년내 연체일수가 30일 이상 된 대출연체 유지 기관수(연체대출금 상환 포함)         0.067179
기준년월 시점 공공정보 (한국신용정보원) 5년내 발생건수.1                           0.065439
기준년월 시점 1년내 연체일수가 30일 이상 된 대출연체 유지 과목수(연체대출금 상환 포함)         0.062899
기준년월 시점 3년내 대출연체 신규발생 과목수(연체대출금 상환 포함)                      0.061732
기준년월 시점 3년내 대출연체 발생분 최장 보유일수(연체대출금 상환 포함)                   0.060434
기준년월 

/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2742: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [32]:
# 상위 15개 피처명 추출
top_features = correlations.head(15).index.tolist()

# X, y 정의
X = df_encoded[top_features]
y = df_encoded['기준일로부터 향후 1년내 부도, 기업회생, 90일이상 금융연체, 대지급 등 발생여부(신용정보원 기준). 1: 향후1년내 부도발생, 0: 향후1년내 부도미발생']


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [34]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'
)
model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [35]:
y_pred = model.predict(X_test)


In [36]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

           0     0.9919    0.8699    0.9269      3089
           1     0.0474    0.4762    0.0862        42

    accuracy                         0.8646      3131
   macro avg     0.5196    0.6730    0.5065      3131
weighted avg     0.9792    0.8646    0.9156      3131



In [37]:
df_fe = df_encoded.copy()

# 재무 비율형 피처 생성
df_fe['부채비율'] = df_fe['부채총계'] / (df_fe['자본총계'] + 1e-6)
df_fe['유동비율'] = df_fe['유동자산'] / (df_fe['유동부채'] + 1e-6)
df_fe['영업이익률'] = df_fe['영업손익'] / (df_fe['매출액'] + 1e-6)
df_fe['순이익률'] = df_fe['당기순이익'] / (df_fe['매출액'] + 1e-6)
df_fe['이자보상배율'] = df_fe['EBIT'] / (df_fe['이자비용'] + 1e-6)


In [38]:
# 증가율 (증가비율)
df_fe['자산총계_증가율'] = (df_fe['자산총계'] - df_fe['자산총계(전기)']) / (df_fe['자산총계(전기)'] + 1e-6)
df_fe['매출액_증가율'] = (df_fe['매출액'] - df_fe['전기매출액']) / (df_fe['전기매출액'] + 1e-6)
df_fe['영업이익_증가율'] = (df_fe['영업손익'] - df_fe['전기영업이익']) / (abs(df_fe['전기영업이익']) + 1e-6)
df_fe['순이익_증가율'] = (df_fe['당기순이익'] - df_fe['당기순이익(전기)']) / (abs(df_fe['당기순이익(전기)']) + 1e-6)

# 단순 변화량 (절대값)
df_fe['자산총계_증가액'] = df_fe['자산총계'] - df_fe['자산총계(전기)']
df_fe['매출액_증가액'] = df_fe['매출액'] - df_fe['전기매출액']
df_fe['영업이익_증가액'] = df_fe['영업손익'] - df_fe['전기영업이익']
df_fe['순이익_증가액'] = df_fe['당기순이익'] - df_fe['당기순이익(전기)']


In [39]:
import numpy as np

log_features = [
    '매출액', '자산총계', '부채총계',
    '유동자산', '비유동자산', '단기차입금',
    '당기순이익', '영업손익'
]

for col in log_features:
    df_fe[f'log_{col}'] = np.log1p(df_fe[col].clip(lower=0))


In [40]:
# 매출액을 기반으로 규모 그룹핑 (0: 소형, 1: 중형, 2: 대형)
df_fe['매출액_구간'] = pd.qcut(df_fe['매출액'], q=3, labels=[0, 1, 2])

# 부채비율을 기준으로 위험도 구간화
df_fe['부채비율_구간'] = pd.cut(df_fe['부채비율'], bins=[-1, 1, 2, 10], labels=[0, 1, 2])

# ROE 구간화 (자기자본이익률)
df_fe['ROE_구간'] = pd.cut(df_fe['자기자본이익률(ROE)'], bins=[-100, 0, 10, 100], labels=[0, 1, 2])


In [41]:
# 부채와 현금흐름을 결합하여 기업의 즉시 상환 능력을 보는 지표
df_fe['부채_현금흐름_비율'] = df_fe['부채총계'] / (df_fe['현금흐름'] + 1e-6)

# 이자보상배율이 낮고, 부채비율이 높다면 위험
df_fe['이자_부채_위험지수'] = (df_fe['부채비율'] > 1).astype(int) * (df_fe['이자보상배율'] < 1).astype(int)

# EBITDA 대비 차입금 비율 (부채 부담도)
df_fe['차입금_EBITDA'] = df_fe['단기차입금'] / (df_fe['EBITDA'] + 1e-6)


In [45]:
from sklearn.preprocessing import LabelEncoder

# object 타입 컬럼 추출
obj_cols = X_train.select_dtypes(include='object').columns

# 라벨 인코딩
le = LabelEncoder()
for col in obj_cols:
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [46]:
# 타깃 변수 지정
y = df['기준일로부터 향후 1년내 부도, 기업회생, 90일이상 금융연체, 대지급 등 발생여부(신용정보원 기준). 1: 향후1년내 부도발생, 0: 향후1년내 부도미발생']

# 피처 변수에서 타깃 열 제외
X = df.drop(columns=[y.name])


In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # 클래스 비율 유지
)


In [48]:
from sklearn.preprocessing import LabelEncoder

# 문자형(object) 컬럼만 추출
obj_cols = X_train.select_dtypes(include='object').columns

# 각 컬럼별로 라벨 인코딩 수행
le = LabelEncoder()
for col in obj_cols:
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [52]:
# 전체 피처 이름 리네이밍 함수 적용
def clean_column_names(df):
    df.columns = [col.replace('(', '_')
                      .replace(')', '_')
                      .replace('.', '_')
                      .replace(':', '_')
                      .replace(' ', '_')
                      .replace('-', '_') for col in df.columns]
    return df

# train/test에 동일하게 적용
X_train = clean_column_names(X_train.copy())
X_test = clean_column_names(X_test.copy())


In [54]:
print(f"X_train shape: {X_train.shape}, len(columns): {len(X_train.columns)}")


X_train shape: (99991, 164), len(columns): 164


In [55]:
# 컬럼 이름 정제 함수
def clean_columns(df):
    df = df.copy()
    df.columns = [str(col).replace('(', '_').replace(')', '_').replace('.', '_')
                         .replace(':', '_').replace(' ', '_').replace('-', '_')
                         .replace('/', '_').replace('\\', '_') for col in df.columns]
    return df

# 정제 적용
X_train = clean_columns(X_train)
X_test = clean_columns(X_test)


In [56]:
assert X_train.shape[1] == len(X_train.columns), "열 수와 컬럼 수 불일치!"
assert X_train.columns.equals(X_test.columns), "train/test 컬럼 불일치!"


In [57]:
# 두 데이터프레임을 합쳐서 get_dummies
combined = pd.concat([X_train, X_test], axis=0)
combined_encoded = pd.get_dummies(combined)

# 다시 분리
X_train = combined_encoded.iloc[:len(X_train)]
X_test = combined_encoded.iloc[len(X_train):]


In [58]:
from lightgbm import LGBMClassifier

# 모델 정의
lgbm = LGBMClassifier(
    random_state=42,
    class_weight='balanced',  # 불균형 클래스 대응
    n_estimators=100,
    max_depth=5
)

# 모델 학습
lgbm.fit(X_train, y_train)

# 예측
y_pred = lgbm.predict(X_test)


[LightGBM] [Info] Number of positive: 1461, number of negative: 98530
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29098
[LightGBM] [Info] Number of data points in the train set: 99991, number of used features: 163
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[LightGBM] [Fatal] Wrong size of feature_names


LightGBMError: Wrong size of feature_names